# Collect user feedback 👍

One final addition to the app is to collect user feedback.

This provides an invaluable source of information for improving the app and understanding how users are interacting with it.

In this notebook, we will add user feedback buttons to the app, connect feedback to corresponding traces, and explore the feedback data from the cloud.

In [ ]:
# For autoreloading external modules
%load_ext autoreload
%autoreload 2

## Extending app with feedback buttons

We will add feedback buttons for each of the outputs in the app, so for the `title`, `summary`, etc.

Having as specific feedback as possible the better, but we also want to keep the feedback process simple and quick.

> **Exercise** 📝
>
> - Explore the provided function `write_and_collect_feedback` in the `app/utils.py` file.
> - Fill in the TODO's `# TODO(13-collect-feedback)` in `components.py` to collect feedback for each model output.
> - Rerun the app locally to check if the buttons were added correctly
> 

The buttons might not work yet, because they're trying to store the feedback associated with the corresponding `trace_id` in the cloud, but we haven't passed the correct `trace_id` along with the feedback yet!

## Logging feedback with trace

Now, it's important that the feedback is associated to the correct trace.

> **Exercise** 📝
>
> - Fill in the TODO's `# TODO(13-feedback-with-trace)` in both `components.py` and `extraction.py` to make sure that feedback is associated with the correct traces.
> - Run the app again, and trigger the feedback buttons so that the feedback is logged in the cloud.
> - Check the Azure Monitor, to see if the logs are being stored correctly.
>
> 💡 Hint:
>
> - Where it might now say `results, _ = extract_info_from_articles(articles)` in `components.py`, you might want to change it to `results, trace_ids = ...` and then store the trace IDs in the streamlit session state, `st.session_state["trace_ids"] = trace_ids`, so they can be used for logging the feedback.

## Loading feedback data from cloud

We now have traces, logs, feedback, on which we can create new metrics and insights in our monitoring dashboard.
We can query downvoted feedback in the Logs Explorer, to collect traces to subsequently inspect in the Trace Explorer.
We can do a lot now!

But not only can we interact with our data in the cloud, we can also load it in and analyse it programmatically.

In [ ]:
from llmops_training.news_reader.logs import load_entries_with_feedback, load_feedback_entries 
from llmops_training.news_reader.extraction import extract_general_info
from llmops_training.news_reader.logs import configure_structlog, configure_tracer

configure_structlog()
configure_tracer()

In [ ]:
downvote_entries = load_feedback_entries("downvote", from_hours_ago=1)
upvote_entries = load_feedback_entries("upvote", from_hours_ago=1)

if downvote_entries:
    print(downvote_entries[0])

In [ ]:
# Note: if this function fails, check the docstring to see how you can solve it.

downvoted_entries = load_entries_with_feedback("downvote", from_hours_ago=1)
upvoted_entries = load_entries_with_feedback("upvote", from_hours_ago=1)

if downvoted_entries:
    print(downvoted_entries[0])

In [ ]:
output = extract_general_info(
    prompt_template="""
    Extract structured information from the following article.
    Give the title and a single sentence summary.
    
    Article: \n{article}
    """,
    article=downvoted_entries[0]["jsonPayload"]["article"],
)

output.title